# XGB(one of the GLM family with Gradient Imp.): 

# Prepare the environment 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.cluster import KMeans
from sklearn.experimental import enable_hist_gradient_boosting
from category_encoders import MEstimateEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import lightgbm as lgbm
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


In [2]:
train = pd.read_csv("../input/30-days-of-ml/train.csv", index_col=0)
test = pd.read_csv("../input/30-days-of-ml/test.csv", index_col=0)
X_test = test.copy()
y = train['target']
X =  train.drop(['target'], axis=1).copy()



cat_cols = [col for col in X.columns if X[col].dtype == 'object']
encoder = MEstimateEncoder(cols=cat_cols, m=5)
X = encoder.fit_transform(X, y)
X_test = encoder.transform(X_test)

kmeans = KMeans(n_clusters=7)
Cluster = kmeans.fit_predict(X)
testCluster = kmeans.fit_predict(X_test)
X["Cluster"] = Cluster
X_test["Cluster"] = testCluster


X_data = X; 
X_test_data  = X_test;
y_data = y; 
del X
del y
del X_test

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [3]:
X_data["Cluster"]

id
1         1
2         5
3         2
4         5
6         2
         ..
499993    2
499996    1
499997    5
499998    4
499999    2
Name: Cluster, Length: 300000, dtype: int32

# Train

In [4]:
X = X_data.copy()
y = y_data.copy()
X_test = X_test_data.copy()

best_para = {'learning_rate': 0.09739156815144875, 'reg_lambda': 1.9383336479859923e-05,
             'reg_alpha': 24.134445292436887, 'subsample': 0.9936134241222736,
             'colsample_bytree': 0.16484879725660215, 'max_depth': 5}

# Define the model
model = XGBRegressor(random_state = 0, n_estimators = 1500, n_jobs =4, **best_para)

rmses = []
model1_pred = np.zeros(train.shape[0])
pred_kf = []
n_folding = 7

kf = KFold(n_splits = n_folding, random_state=0, shuffle=True)

for ids in kf.split(X, y):
    train_ids, valid_ids = ids[0], ids[1]
    
    X_train = X.iloc[train_ids]
    y_train = y.iloc[train_ids]
    X_valid = X.iloc[valid_ids]
    y_valid = y.iloc[valid_ids]
 
    # Fit the model
    model.fit(X_train, y_train, early_stopping_rounds = 200, eval_set=[(X_valid, y_valid)], verbose = False)
    
    #validation
    model1_pred[valid_ids] = model.predict(X_valid)
    
    y_pred = model1_pred[valid_ids]
    
    rmse = mean_squared_error(y_pred, y_valid)
    
    rmses.append(rmse)
    
    #predictions
    pred = model.predict(X_test)
    pred_kf.append(pred)   

    

scores = [1 - rmse for rmse in rmses] 
print(scores)
model1_test_pred = [pred_kf[i]*scores[i]/sum(scores) for i in range(n_folding)]
model1_test_pred = np.sum(model1_test_pred, axis=0)

print(rmses)
print("validation rmse:", np.mean(rmses), "STD:", np.std(rmses))



[0.4857849424540662, 0.4823896693172689, 0.4889433600199643, 0.4792971117613113, 0.4878991111132257, 0.48873476725460063, 0.49033053381754943]
[0.5142150575459338, 0.5176103306827311, 0.5110566399800357, 0.5207028882386887, 0.5121008888867743, 0.5112652327453994, 0.5096694661824506]
validation rmse: 0.5138029291802876 STD: 0.0037079076903382387


In [5]:
predictions = model1_test_pred

# Submit

In [6]:
# Save the predictions to a CSV file
output = pd.DataFrame({'Id': X_test.index,
                       'target': predictions})
output.to_csv('submission.csv', index=False)
output

,Id,target
0,0,8.083234
1,5,8.391595
2,15,8.419264
3,16,8.507890
4,17,8.110280
...,...,...
199995,499987,8.062738
199996,499990,8.439714
199997,499991,8.520163
199998,499994,8.160011
